In [1]:
import multiprocessing
from concurrent.futures import ProcessPoolExecutor

from tqdm.auto import tqdm
from utils import (
    ARDSchema,
    ExperimentSchema,
    OptimizationSchema,
    create_groups,
    get_group,
    load_ard_data,
    optimize_single,
    plot_2outputs,
    postproc_mw_events,
    run_experiment,
    run_mowing,
)

In [2]:
multiprocessing.set_start_method("fork")

In [3]:
ard_config = ARDSchema(
    s1_path="./data/S1COH6D/signals/dataset/",
    s2_path="./data/S2/signals-crop/dataset/",
    otsc_path="./data/Sinergise OTSC results 2020.xlsx",
    min_foi_nobs=15,
    min_foi_size=50,
    lu_label="5",
)

# dataframes for meadows
df_s1, df_s2 = load_ard_data(ard_config)
_, df_s2_mowing = load_ard_data(ARDSchema(**{**ard_config.dict(), "s2_valid_query_mode": "strict"}))

In [4]:
# run mowing for NDVI
df_list = [df for _, df in df_s2_mowing.groupby("POLY_ID")]

with ProcessPoolExecutor() as executor:
    result_list = list(tqdm(executor.map(run_mowing, df_list), total=df_s2_mowing.POLY_ID.nunique()))

results = []
for result in result_list:
    results.extend(result)

df_mw = postproc_mw_events(results)

  0%|          | 0/47135 [00:00<?, ?it/s]

In [5]:
# dump data
df_s1.to_parquet("./data/s1_ard.pq")
df_s2.to_parquet("./data/s2_ard.pq")
df_mw.to_parquet("./data/s2_mowing.pq")